In [9]:
import os
import random

def find_mismatched_folders(base_image_dir, base_label_dir):
    """
    이미지와 라벨 디렉토리 간에 일치하지 않는 폴더를 탐지하고 출력합니다.
    """
    # 디렉토리 이름 목록 가져오기 (숨겨진 폴더 무시, 대소문자 및 공백 처리)
    image_folders = {folder.strip().lower() for folder in os.listdir(base_image_dir) if not folder.startswith(".") and os.path.isdir(os.path.join(base_image_dir, folder))}
    label_folders = {folder.strip().lower() for folder in os.listdir(base_label_dir) if not folder.startswith(".") and os.path.isdir(os.path.join(base_label_dir, folder))}

    # 교집합 및 차집합 계산
    common_folders = image_folders & label_folders
    only_in_images = image_folders - label_folders
    only_in_labels = label_folders - image_folders

    # 출력
    if only_in_images:
        print("Folders only in images:")
        for folder in sorted(only_in_images):
            print(f"  {folder}")

    if only_in_labels:
        print("Folders only in labels:")
        for folder in sorted(only_in_labels):
            print(f"  {folder}")

    if not only_in_images and not only_in_labels:
        print("All folders are matched between images and labels.")

    return common_folders

def keep_limited_files(image_dir, label_dir, max_count=1500):
    """
    이미지와 라벨 디렉토리에서 교집합만 남긴 후, 최대 max_count만 유지하고 초과 파일 삭제.
    """
    # 이미지와 라벨 파일 이름 추출 (대소문자 및 공백 처리)
    image_names = {os.path.splitext(file)[0].strip().lower() for file in os.listdir(image_dir) if file.endswith((".jpg", ".png", ".jpeg"))}
    label_names = {os.path.splitext(file)[0].strip().lower() for file in os.listdir(label_dir) if file.endswith(".txt")}

    # 교집합 계산
    common_names = sorted(image_names & label_names)

    # 교집합 개수 확인 및 초과 파일 삭제
    if len(common_names) > max_count:
        files_to_keep = set(random.sample(common_names, max_count))
        files_to_remove = set(common_names) - files_to_keep

        # 삭제 로직
        for file_name in files_to_remove:
            for ext in [".jpg", ".jpeg", ".png"]:
                img_path = os.path.join(image_dir, f"{file_name}{ext}")
                if os.path.exists(img_path):
                    os.remove(img_path)
                    print(f"Deleted image file: {img_path}")

            label_path = os.path.join(label_dir, f"{file_name}.txt")
            if os.path.exists(label_path):
                os.remove(label_path)
                print(f"Deleted label file: {label_path}")

        print(f"Reduced {os.path.basename(image_dir)} to {max_count} items.")
    else:
        print(f"{os.path.basename(image_dir)} has {len(common_names)} items, no reduction needed.")

def process_all_folders(base_image_dir, base_label_dir, max_count=1500):
    """
    모든 폴더에 대해 교집합만 남기고, max_count개로 제한.
    """
    # 일치하지 않는 폴더 찾기
    common_folders = find_mismatched_folders(base_image_dir, base_label_dir)

    # 공통 폴더 처리
    for folder in sorted(common_folders):
        image_dir = os.path.join(base_image_dir, folder)
        label_dir = os.path.join(base_label_dir, folder)

        if os.path.isdir(image_dir) and os.path.isdir(label_dir):
            print(f"Processing {folder}...")
            keep_limited_files(image_dir, label_dir, max_count=max_count)

# 경로 설정
base_image_dir = os.path.expanduser("~/yolov7/food_data_yai/images/train")
base_label_dir = os.path.expanduser("~/yolov7/food_data_yai/labels/train")

# 실행
process_all_folders(base_image_dir, base_label_dir)


All folders are matched between images and labels.
Processing rice...
rice has 895 items, no reduction needed.


In [3]:
import os

def count_files_in_folders(base_image_dir, base_label_dir):
    """
    각 train 폴더 내 이미지와 txt 파일 개수를 출력합니다.
    
    Parameters:
    - base_image_dir (str): 이미지 디렉토리 경로
    - base_label_dir (str): 라벨 디렉토리 경로
    """
    # 모든 폴더 이름 가져오기
    image_folders = sorted(folder for folder in os.listdir(base_image_dir) if os.path.isdir(os.path.join(base_image_dir, folder)))
    label_folders = sorted(folder for folder in os.listdir(base_label_dir) if os.path.isdir(os.path.join(base_label_dir, folder)))

    print(f"{'Folder':<20} {'Images':<10} {'Labels':<10}")
    print("=" * 40)

    # 모든 폴더에 대해 이미지와 라벨 파일 개수 출력
    all_folders = sorted(set(image_folders) | set(label_folders))
    for folder in all_folders:
        image_folder_path = os.path.join(base_image_dir, folder)
        label_folder_path = os.path.join(base_label_dir, folder)

        # 이미지 파일 개수
        image_count = len([file for file in os.listdir(image_folder_path) if file.endswith((".jpg", ".png", ".jpeg"))]) if os.path.exists(image_folder_path) else 0

        # 라벨 파일 개수
        label_count = len([file for file in os.listdir(label_folder_path) if file.endswith(".txt")]) if os.path.exists(label_folder_path) else 0

        print(f"{folder:<20} {image_count:<10} {label_count:<10}")

# 경로 설정
base_image_dir = os.path.expanduser("~/yolov5/food_data_yai/images/train")
base_label_dir = os.path.expanduser("~/yolov5/food_data_yai/labels/train")

# 실행
count_files_in_folders(base_image_dir, base_label_dir)


Folder               Images     Labels    
.ipynb_checkpoints   0          0         
Almond               1499       1499      
Avocado              717        717       
Bacon                1500       1500      
Baguette             1500       1500      
Banana               740        740       
Basil                730        730       
Bell_pepper          667        667       
Bread                1500       1500      
Broccoli             667        667       
Butter               1500       1500      
Cabbage              1500       1500      
Cabbage_kimchi       147        147       
Canned_tuna          383        383       
Carrot               1500       1500      
Celery               818        818       
Cheese               825        825       
Cherry_tomatoes      1500       1500      
Chicken              1500       1500      
Chicken_breast       1500       1500      
Chili_pepper         770        770       
Chili_powder         804        804       
Chili_sauce

In [ ]:
import os
from PIL import Image

def count_corrupted_images_by_class(base_image_dir):
    """
    각 클래스 폴더에 있는 손상된 이미지 파일 개수를 계산합니다.

    Parameters:
    - base_image_dir (str): 이미지 파일이 포함된 기본 디렉토리 경로.

    Returns:
    - dict: 클래스 이름을 키로 하고, 손상된 이미지 수를 값으로 하는 딕셔너리.
    """
    corrupted_counts = {}

    # 각 클래스 폴더 탐색
    for class_name in os.listdir(base_image_dir):
        class_dir = os.path.join(base_image_dir, class_name)
        if os.path.isdir(class_dir):  # 클래스 폴더인지 확인
            corrupted_count = 0
            for file_name in os.listdir(class_dir):
                if file_name.endswith((".jpg", ".jpeg", ".png")):  # 이미지 파일 필터링
                    file_path = os.path.join(class_dir, file_name)
                    try:
                        with Image.open(file_path) as img:
                            img.verify()  # 이미지 검증
                    except (IOError, SyntaxError):  # 손상된 이미지 발견
                        corrupted_count += 1
            corrupted_counts[class_name] = corrupted_count
    
    return corrupted_counts


# 기본 이미지 디렉토리 설정
base_image_dir = "/home/elicer/yolov7/food_data_yai/images/train"

# 손상된 이미지 수 계산
corrupted_counts = count_corrupted_images_by_class(base_image_dir)

# 결과 출력
print("\n클래스별 손상된 이미지 개수:")
for class_name, count in corrupted_counts.items():
    print(f"클래스: {class_name}, 손상된 이미지: {count}장")
